# 装饰器
装饰器用于在源码“标记”函数，以某种方式增强函数的行为。装饰器是可调用的对象，其参数是另一个函数(被装饰的函数)。例如：
```python
@decorate
def target():
    print("running target")
```
它的行为和下列代码一致：
```python
def target():
    print ("running target")
    
target = decorate(target)
```
装饰器通常接受函数后对其进行一些额外操作，而后返回该函数或者将其替换为另一个函数并返回。

In [5]:
def deco(func):
    def inner():
        print ("running inner")
    return inner                             # 返回值为函数：inner

@deco
def target():
    print ("running target()")
target()                           # 输出的却是inner，因为此时target已经变成了inner的引用。

running inner


### 装饰器的执行
Python代码中处处可见 `if __name__ == '__main__:'`的写法，目的就是为了保证代码是在**明确运行时执行**，而不是**作为模块导入时执行。**
而Python装饰器的一个关键特性就是，他们在被装饰的函数定义之后立即运行，也就是说即使当做模块导入，装饰器还是会执行。


### 变量作用域
每个变量能起作用的范围是有限的，比如变量在函数内定义，那它的作用域就只是这个函数；如果在程序主体内定义，那它就是一个全局变量，类似于JS中`var`和`let`的区别。

In [6]:
def f1(a):
    print(a)
    print(b)
    
b = 6
f1(3)

3
6


函数可以取得b的值，这点很正常，b是一个全局变量。

In [7]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

居然失败了！b不是全局变量么？问题在于我们在函数中对b进行了赋值，Python解释器把b当做局部变量了。Python假定在函数内赋值的变量为局部变量。如果想要更改这个行为，可以用global声明。

In [8]:
b = 6
def f2(a):
    global b
    print(a)
    print(b)
    b = 9
    
f2(3)

3
6


### 闭包
**闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。关键在于它能访问定义体之外的非全局变量。**

In [18]:
def make_average():
    series = []
    
    def average(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return average

avg = make_average()
print(avg(10))
print(avg(12))

10.0
11.0


执行过一次函数后，我们第一次添加的10.0的值还在series中，可make_average函数已经返回了啊，它的作用域也已经消失了。

这里就出现了闭包，在average函数中，series是*自由变量(free variable)*，指未在本地作用域绑定的变量。

In [20]:
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


通过Python中的`__code__`属性可以查看函数的变量，如上所示，`series`是一个自由变量。它保存在返回的avg函数的`__closure__`属性中。`__closure__`中的各个元素对应`avg.__code__.co_freevars`中的一个名称。这些元素是一个cell对象，通过元素的cell_contents属性可以查看真正的值。

In [27]:
avg.__closure__[0].cell_contents

[10, 12]

### nonlocal声明
上例的函数不太完美，每次调用都需要重新计算列表的所有元素之和，更好的办法是只存储个数和目前的和。

In [1]:
def make_average():
    count = 0
    total = 0
    
    def average(new_value):
        count += 1
        total += new_value
        return total/count
    return average

avg = make_average()
avg.__code__.co_varnames

('new_value', 'count', 'total')

这样写效率会高很多，但调用却会失败。。原因很简单，就是我们在函数内更改了count和total的值，Python把它们两个当做`average`函数的本地变量了。那怎么处理呢？如果count和total是全局变量，我们可以用globle关键字，但count和total并不是，它们是make_average函数的局部变量。

Python3新引入了nonlocal关键字，可以把变量标记为自由变量，可以解决这个问题。

In [29]:
def make_average():
    count = 0
    total = 0
    
    def average(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return average

avg = make_average()
avg(10)

10.0

## 标准库中的装饰器
**functools.lru_cache做备忘**

functools.lru_cache实现了备忘(memoizaion)功能，把耗时的函数的结果保存起来，传入相同参数时直接使用缓存的结果，避免再次运算。LRU:Last Recently Used的缩写。下边拿经典的指数复杂度的递归斐波那契数列方法举例子：

In [31]:
import functools

@functools.lru_cache()                                # 必须像常规函数一样调用lru_cache，加括号！
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n-1) + fib(n-2)
fib(6)

8

**单分派泛函数**

由于Python不支持函数重载，所以当我们需要根据参数的不同定义不同行为时，通常会使用if/else判断。但这种方法并不佳。我们可以使用Python3.4新增的functools.singledispatch装饰器将整体方案拆分，形成类似重载的效果。

In [38]:
from functools import singledispatch
from collections import abc

@singledispatch                               #为其他函数不处理的基本类型定义处理方法
def total(obj):
    return "hello, int to you " + str(obj)

@total.register(str)                         # 使用@base_funciton.register(type)方法注册
def _(text):                                 # 函数名不重要，什么都可以
    return "hello, i'm str"

@total.register(tuple)
@total.register(abc.MutableSequence)         # 最好处理抽象基类，如number.Integral这些
def _(seq):
    return "Hello, i'm mutablesequence"

print (total(5))
print (total('5'))

hello, int to you 5
hello, i'm str


### 叠加装饰器
一个函数可以被多个装饰器所装饰，注意装饰器的顺序非常重要！例如如下代码：
```python
@d1
@d2
def f():
    print('f')
```
等同于：`f = d1(d2(f))`

### 参数化装饰器
装饰器除了接收被装饰的函数作为第一个参数，还可以接收其他参数。如何做到？创建一个装饰器工厂函数，把参数传给它，返回一个装饰器，然后把它应用到要装饰的函数上。

In [15]:
def param_deco(choose=False):                 # 装饰器工厂函数
    def decorate(func):                       # 装饰器
        if choose:                            # 闭包！choose从定义体外获得，非全局变量   
            print("choosed!")
        else:
            print ("Not choose.")
        return func                           # 装饰器返回一个函数
    return decorate                           # 装饰器工厂函数返回装饰器

@param_deco(choose=True)                      
def f1():
    print("You are choosed!")
    
@param_deco()                                 # 现在即使不传入参数，也要当做函数调用，加括号
def f2():
    print("You are not choosed. Sorry for you.")

choosed!
Not choose.
